# Part E: Observability Check

In this notebook, we will perform an observability analysis for a system described by its state-space representation. We define observability for various output matrices and determine if the system states can be observed from these outputs.

In [1]:
! pip install control
import numpy as np
import pandas as pd
from IPython.display import HTML

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def observability_matrix(A, C):
    """
    Compute the observability matrix for the system (A, C).
    """
    obs_matrix = C
    for i in range(1, A.shape[0]):
        obs_matrix = np.vstack((obs_matrix, C @ np.linalg.matrix_power(A, i)))
    return obs_matrix


In [3]:
# Define system parameters
M = 1000
mass_1 = 100
mass_2 = 100
length_1 = 20
length_2 = 10
g = 9.81

# Define state space matrices A and B with the assigned values
A = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, -(mass_1*g)/M, 0, -(mass_2*g)/M, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, -((M+mass_1)*g)/(M*length_1), 0, -(mass_2*g)/(M*length_1), 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, -(mass_1*g)/(M*length_2), 0, -(g*(M+mass_2))/(M*length_2), 0]
])
print("A: ")
A

A: 


array([[ 0.     ,  1.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     , -0.981  ,  0.     , -0.981  ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  1.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     , -0.53955,  0.     , -0.04905,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  1.     ],
       [ 0.     ,  0.     , -0.0981 ,  0.     , -1.0791 ,  0.     ]])

In [4]:
# Defining the B matrix
B = np.array([[0], [1/M], [0], [1/(M*length_1)], [0], [1/(M*length_2)]])
print("B: ")
B

B: 


array([[0.e+00],
       [1.e-03],
       [0.e+00],
       [5.e-05],
       [0.e+00],
       [1.e-04]])

## Defining Output Matrices

We define different output matrices `C` to observe various combinations of the system's states. The observability of the system is determined for each case.


In [5]:
# Define different C matrices for various outputs
C_matrices = {
    r'x': np.array([[1, 0, 0, 0, 0, 0]]),
    r'theta_1 and theta_2': np.array([[0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0]]),
    r'x and theta_2': np.array([[1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0]]),
    r'x, theta_1 and theta_2': np.array([[1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0]])
}

In [6]:
# Prepare data for the table
table_data = []

# Checking the observability for each C matrix and collecting data for the table
for output, C_matrix in C_matrices.items():
    O_matrix = observability_matrix(A, C_matrix)
    rank = np.linalg.matrix_rank(O_matrix)
    observable = "Yes" if rank == 6 else "No"
    matrix_string = '[' + '; '.join(' '.join(str(int(num)) for num in row) for row in C_matrix) + ']'
    table_data.append({
        'Outputs': output,
        'C Matrix': matrix_string,
        'Rank': rank,
        'Observable': observable
    })

# Create a pandas DataFrame
df = pd.DataFrame(table_data)
# Convert the DataFrame to HTML and allow raw HTML and LaTeX rendering
html = df.to_html(escape=False, index=False)
display(HTML(html))

Outputs,C Matrix,Rank,Observable
x,[1 0 0 0 0 0],6,Yes
theta_1 and theta_2,[0 0 1 0 0 0; 0 0 0 1 0 0],4,No
x and theta_2,[1 0 0 0 0 0; 0 0 0 0 1 0],6,Yes
"x, theta_1 and theta_2",[1 0 0 0 0 0; 1 0 1 0 0 0; 0 0 0 0 1 0],6,Yes
